In [1]:
import cymetric as cym
from cymetric import timeseries
import matplotlib as plt 
import pandas as pd
import sys 
import output as oup
sys.path.insert(0, '../../transition-scenarios/scripts/')
import transition_plots as tp
import numpy as np

/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.enrichment is not yet QA compliant.
  return f(*args, **kwds)


In [26]:
from cymetric import fco_metrics
from cymetric import filters
from cymetric import timeseries

In [2]:
starter_string = 'CT'
scenario_nums = ['0','1','2']

In [3]:
df = oup.initialize_df(scenario_index=starter_string,
                                 scenarios_nums=scenario_nums)

In [4]:
df['Final HLW'] = 0
df['Final Depleted U'] = 0 
df['Total uranium ore'] = 0
df['Total idle capacity'] = 0
df['Last date idle capacity'] = 0
df['Duration of transition'] = 0

In [5]:
df

,Final HLW,Final Depleted U,Total uranium ore,Total idle capacity,Last date idle capacity,Duration of transition
CT,,,,,,
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0


In [6]:
output_start = '../cyclus-files/oat/cooling-time/ct'
ev_dict = {}
for x in range(len(scenario_nums)): 
    output_file = output_start + scenario_nums[x]+'.sqlite'
    ev_dict[scenario_nums[x]] = cym.Evaluator(db=cym.dbopen(output_file),write=True)

In [39]:
for x in range(len(scenario_nums)): 
    #df.loc[scenario_nums[x],'Final HLW'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['lwrreprocessingwaste','moxreprocessingwaste','frreprocessingwaste'])['Mass'].cumsum().iloc[-1]
    #df.loc[scenario_nums[x],'Final Depleted U'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['enrichmentwaste'])['Mass'].cumsum().max()
    #df.loc[scenario_nums[x],'Total uranium ore'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['sourceout'])['Mass'].cumsum().max()
    df.loc[scenario_nums[x],'Total idle capacity'], df.loc[scenario_nums[x],'Last date idle capacity'], udict = idlecap(ev_dict[scenario_nums[x]],'(60000+250*t/12)/1000')
    
    

In [40]:
df

,Final HLW,Final Depleted U,Total uranium ore,Total idle capacity,Last date idle capacity,Duration of transition
CT,,,,,,
0,6.699047e+06,5.658634e+08,1.437000e+11,13840.448680,1439,0
1,1.253578e+07,7.807626e+08,1.437000e+11,139.083333,962,0
2,1.245817e+07,7.807626e+08,1.437000e+11,139.083333,962,0


In [42]:
def idlecap(evaluator,demand_eq): 
    ep = evaluator.eval('FcoMonthlyElectricityGenerated')
    t = np.arange(0,1440)
    power = eval(demand_eq)
    diff = list(ep['Energy'])-power
    total = 0
    undersupply_times = {}
    for x in range(len(diff)):
        if diff[x] < 0: 
            total -= diff[x]
            undersupply_times[x] = -diff[x]
    last_undersupply = max(k for k, v in undersupply_times.items())
    return total, last_undersupply, undersupply_times

In [44]:
def sensitivity(base_case,init_df):
    """ This function takes a dataframe 
    """
    SA_df = init_df.copy()
    M = init_df.index.size
    categories=list(init_df)
    N = len(categories)
    row = 0
    for x in range(M): 
        if init_df.index[x] == base_case: 
            basecase_index = row
        row += 1
    for x in range(M): 
        if init_df.index[x] == base_case: 
            for y in range(N): 
                SA_df.iloc[x,y] = 0
        else: 
            for y in range(N): 
                if float(init_df.iloc[basecase_index,y]) == 0: 
                    SA_df.iloc[x,y] = np.nan
                else:
                    SA_df.iloc[x,y] = (init_df.iloc[x,y]-init_df.iloc[basecase_index,y])/init_df.iloc[basecase_index,y]*100  
    return SA_df

In [46]:
sensitivity(base_case='1',init_df=df)

,Final HLW,Final Depleted U,Total uranium ore,Total idle capacity,Last date idle capacity,Duration of transition
CT,,,,,,
0,-46.560567,-27.524262,0.0,9851.191382,49.5842,NaN
1,0.000000,0.000000,0.0,0.000000,0.0000,0.0
2,-0.619034,0.000000,0.0,0.000000,0.0000,NaN


In [48]:
df_p = oup.initialize_df(scenario_index=starter_string,
                                 scenarios_nums=scenario_nums)

In [49]:
df_p['Max Pu in LWR CP'] = 0
df_p['Max Pu in MOX CP'] = 0 
df_p['Max Pu in SFR CP'] = 0
df_p['Max Pu in all CP'] = 0
df_p['Max Pu in HLW'] = 0
df_p['Max Pu in all RPR'] = 0

In [50]:
df_p

,Max Pu in LWR CP,Max Pu in MOX CP,Max Pu in SFR CP,Max Pu in all CP,Max Pu in HLW,Max Pu in all RPR
CT,,,,,,
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0


In [52]:
for x in range(len(scenario_nums)): 
    df_p.loc[scenario_nums[x],'Max Pu in LWR CP'] = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrstorage'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity'].max()
    df_p.loc[scenario_nums[x],'Max Pu in MOX CP'] = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['moxstorage'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity'].max()
    df_p.loc[scenario_nums[x],'Max Pu in SFR CP'] = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['frstorage'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity'].max()
    df_p.loc[scenario_nums[x],'Max Pu in all CP'] = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrstorage','moxstorage','frstorage'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity'].max()
    df_p.loc[scenario_nums[x],'Max Pu in HLW'] = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['enrichmentsink','lwrsink','moxsink','frsink'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity'].max()
    df_p.loc[scenario_nums[x],'Max Pu in all RPR'] = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrreprocessing','moxreprocessing','frreprocessing'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity'].max()  

TypeError: 'NoneType' object is not subscriptable

In [ ]:
df_p